In [8]:
from keplergl import KeplerGl
import numpy as np
import pandas as pd
import geopandas as gpd
import glob
import os
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

In [9]:
# Caminhos para os shapefiles
shapefile_path = '804387637e9638887b3bb625354623c5c83526532fac778ce885584328e169e4-1719798137525.shp' 
districts_shapefile_path = 'distrito/distrito.shp' 

# Carregar o shapefile das arestas
gdf_edges = gpd.read_file(shapefile_path)

# Carregar o shapefile dos distritos
districts_gdf = gpd.read_file(districts_shapefile_path)
districts_gdf = districts_gdf.set_crs(epsg=31983)

# Garantir que os CRS são iguais
gdf_edges = gdf_edges.to_crs(districts_gdf.crs)

# Carregar e processar os dados CSV
caminho_pasta = "csvs"
arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))

lista_dataframes = []
for arquivo in arquivos_csv:
    temp_df = pd.read_csv(arquivo, parse_dates=['hour'])
    lista_dataframes.append(temp_df)

df_all = pd.concat(lista_dataframes, ignore_index=True)   

In [10]:
# Extrair componentes de data e hora
df_all['date'] = pd.to_datetime(df_all['hour'])
df_all['year'] = df_all['date'].dt.year
df_all['month'] = df_all['date'].dt.month
df_all['day_of_week'] = df_all['date'].dt.dayofweek
df_all['hour_of_day'] = df_all['date'].dt.hour

# Calcular os totais para gêneros e faixas etárias
df_all['male_people_count'] = df_all['forward_male_people_count'].fillna(0) + df_all['reverse_male_people_count'].fillna(0)
df_all['female_people_count'] = df_all['forward_female_people_count'].fillna(0) + df_all['reverse_female_people_count'].fillna(0)
df_all['unspecified_people_count'] = df_all['forward_unspecified_people_count'].fillna(0) + df_all['reverse_unspecified_people_count'].fillna(0)

df_all['18_34_people_count'] = df_all['forward_18_34_people_count'].fillna(0) + df_all['reverse_18_34_people_count'].fillna(0)
df_all['35_54_people_count'] = df_all['forward_35_54_people_count'].fillna(0) + df_all['reverse_35_54_people_count'].fillna(0)
df_all['55_64_people_count'] = df_all['forward_55_64_people_count'].fillna(0) + df_all['reverse_55_64_people_count'].fillna(0)
df_all['65_plus_people_count'] = df_all['forward_65_plus_people_count'].fillna(0) + df_all['reverse_65_plus_people_count'].fillna(0)

df_all['commute_trip_count'] = df_all['forward_commute_trip_count'].fillna(0) + df_all['reverse_commute_trip_count'].fillna(0)
df_all['leisure_trip_count'] = df_all['forward_leisure_trip_count'].fillna(0) + df_all['reverse_leisure_trip_count'].fillna(0)
df_all['total_trip_count'] = df_all['commute_trip_count'] + df_all['leisure_trip_count']

# Mapear números dos meses para nomes em português
month_names = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho',
    7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

In [11]:
activity_type_widget = widgets.Dropdown(
    options=['Todos', 'Deslocamento', 'Lazer'],
    value='Todos',
    description='Tipo de Atividade:'
)

ano_widget = widgets.SelectMultiple(
    options=[2019, 2020, 2021, 2022, 2023, 2024],
    value=[2019, 2020, 2021, 2022, 2023, 2024],
    description='Ano:'
)

mes_widget = widgets.SelectMultiple(
    options=[('Janeiro', 1), ('Fevereiro', 2), ('Março', 3), ('Abril', 4), ('Maio', 5), ('Junho', 6),
             ('Julho', 7), ('Agosto', 8), ('Setembro', 9), ('Outubro', 10), ('Novembro', 11), ('Dezembro', 12)],
    value=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    description='Mês:'
)

genero_widget = widgets.Dropdown(
    options=['Todos', 'Homem', 'Mulher', 'Não especificado'],
    value='Todos',
    description='Gênero:'
)

faixa_etaria_widget = widgets.Dropdown(
    options=['Todos', '18-34', '35-54', '55-64', '65+'],
    value='Todos',
    description='Faixa Etária:'
)

map_genero = {
    'Todos': 'total',
    'Homem': 'male',
    'Mulher': 'female',
    'Não especificado': 'unspecified'
}

map_atividade = {
    'Todos': 'total',
    'Deslocamento': 'commute',
    'Lazer': 'leisure'
}

map_faixa_etaria = {
    'Todos': 'total',
    '18-34': '18_34',
    '35-54': '35_54',
    '55-64': '55_64',
    '65+': '65_plus'
}

Aplicar Filtros

In [12]:
def update_df(ano, mes, genero, faixa_etaria, activity_type):
    df_filtered = df_all.copy()
    
    # Filtrar pelos anos e meses selecionados
    df_filtered = df_filtered[df_filtered['year'].isin(ano)]
    df_filtered = df_filtered[df_filtered['month'].isin(mes)]
    
    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return pd.DataFrame()
    
    # Mapear as opções selecionadas para os valores correspondentes
    activity_type_mapped = map_atividade[activity_type]
    genero_mapped = map_genero[genero]
    faixa_etaria_mapped = map_faixa_etaria[faixa_etaria]
    
    # Selecionar a coluna de contagem de viagens apropriada
    if activity_type_mapped == 'total':
        trip_count_column = 'total_trip_count'
    else:
        trip_count_column = activity_type_mapped + '_trip_count'
    
    # Tratar a seleção de gênero
    if genero_mapped != 'total':
        df_filtered['people_count'] = df_filtered[genero_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['people_count'] = df_filtered[['male_people_count', 'female_people_count', 'unspecified_people_count']].fillna(0).sum(axis=1)
    
    # Tratar a seleção de faixa etária
    if faixa_etaria_mapped != 'total':
        df_filtered['age_group_count'] = df_filtered[faixa_etaria_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['age_group_count'] = df_filtered[['18_34_people_count', '35_54_people_count', '55_64_people_count', '65_plus_people_count']].fillna(0).sum(axis=1)
    
    # Filtrar dados onde as contagens de pessoas e faixa etária são maiores que zero
    df_filtered = df_filtered[(df_filtered['people_count'] > 0) & (df_filtered['age_group_count'] > 0)]
    
    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return pd.DataFrame()
    
    # Agrupar os dados por 'edge_uid' e somar as contagens de viagens
    df_grouped = df_filtered.groupby('edge_uid', as_index=False)[trip_count_column].sum()
    df_grouped['edge_uid'] = df_grouped['edge_uid'].astype('int32')
    df_grouped.rename(columns={trip_count_column: 'total_trip_count'}, inplace=True)
    
    return df_grouped

In [ ]:
def plot_bar(df_grouped, ano, activity_type, max_bars=10):
    if df_grouped.empty:
        return
    
    # Mesclar os dados agregados com o GeoDataFrame das arestas
    merged_gdf = gdf_edges.merge(df_grouped, left_on='edgeUID', right_on='edge_uid')
    
    # Realizar interseção espacial para obter apenas as arestas dentro dos distritos
    merged_gdf = gpd.sjoin(merged_gdf, districts_gdf[['geometry', 'ds_nome']], how='left', predicate='intersects')
    
    # Agrupar por distrito
    totals_per_district = merged_gdf.groupby('ds_nome', as_index=False)['total_trip_count'].sum()
    # Remover linhas com valores nulos em 'ds_nome'
    totals_per_district = totals_per_district.dropna(subset=['ds_nome'])
    # Ordenar os dados
    totals_per_district = totals_per_district.sort_values('total_trip_count', ascending=False)
    
    if max_bars is not None:
        totals_per_district = totals_per_district.head(max_bars)
    
    plt.figure(figsize=(18, 7))
    plt.bar(totals_per_district['ds_nome'], totals_per_district['total_trip_count'], label='Total Viagens', width=0.5)
    plt.title(f'Total de Viagens por Distrito ({min(ano)} a {max(ano)}) - Atividade: {activity_type}', fontsize=20)
    plt.xlabel('Distrito', labelpad=40)
    plt.ylabel('Total de Viagens', fontsize=20)
    plt.xticks(rotation=45, ha='right', fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(fontsize=20)
    plt.tight_layout()
    
    # Salvar o gráfico
    output_dir = 'graficos-distritos'
    os.makedirs(output_dir, exist_ok=True)
    file_name = f"grafico_viagens_distritos_{activity_type}_{min(ano)}_{max(ano)}.png"
    file_name = file_name.replace(' ', '_').replace(',', '')  
    file_path = os.path.join(output_dir, file_name)
    plt.savefig(file_path)
    
    plt.show()

In [14]:
def interactive_plot(activity_type, genero, faixa_etaria, ano, mes):
    df_grouped = update_df(ano, mes, genero, faixa_etaria, activity_type)
    plot_bar(df_grouped, ano, activity_type)

interact(
    interactive_plot,
    activity_type=activity_type_widget,
    genero=genero_widget,
    faixa_etaria=faixa_etaria_widget,
    ano=ano_widget,
    mes=mes_widget
)

interactive(children=(Dropdown(description='Tipo de Atividade:', options=('Todos', 'Deslocamento', 'Lazer'), v…

<function __main__.interactive_plot(activity_type, genero, faixa_etaria, ano, mes)>